In [14]:
from tqdm import tqdm
import networkx as nx
from copy import deepcopy as cp
import os
import io
import numpy as np
import json

In [5]:
import stanza
parser = stanza.Pipeline(lang='ru', processors='tokenize,lemma')
def load_testing_sample(fname, normalize = True):
    with io.open(fname, "r", encoding='utf-8') as f:
        text_ = f.read()
    sentences = text_.split("\n")

    sentences_without_breaks = []
    true_segment_breaks = []
    prev_sent = None
    for sent in sentences:
        if sent.strip() != "":
            if sent != "--- SEGMENT BRAKE ---":
                if normalize:
                    sent_norm_text = ""
                    try:
                        doc = parser(sent).sentences[0]
                        for elm in doc.words:
                            sent_norm_text += elm.lemma + ' '
                    except IndexError as err:# это встретилось пустое предложение
                        raise err
                else:
                    sent_norm_text = sent
                sentences_without_breaks.append(sent_norm_text)
        
                if prev_sent is not None:
                    if prev_sent == "--- SEGMENT BRAKE ---":
                        true_segment_breaks.append(True)
                    else:
                        true_segment_breaks.append(False)
            prev_sent = sent
    true_segment_breaks = np.array(true_segment_breaks)
    return true_segment_breaks, sentences_without_breaks

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-16 13:42:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-16 13:42:33 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2024-10-16 13:42:34 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| lemma     | syntagrus_nocharlm |

2024-10-16 13:42:34 INFO: Using device: cpu
2024-10-16 13:42:34 INFO: Loading: tokenize
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: 

In [11]:
texts_directory = "segmentation_dataset"

all_true_breaks = []
all_sentences_without_breaks = []
for fname in tqdm(os.listdir(texts_directory)):
    path = texts_directory + "/" + fname
    true_breaks, sentences_without_breaks = load_testing_sample(path, normalize = True)
    all_true_breaks.append(true_breaks)
    all_sentences_without_breaks.append(sentences_without_breaks)

100%|██████████| 64/64 [01:04<00:00,  1.00s/it]


In [17]:
with open("classificator.json",'r') as f:
    classificator = json.load(f)

In [20]:
from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [30]:
v1 = navec['кот'].reshape(1, -1)
v2 = navec['кто'].reshape(1, -1)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
cosine_similarity(v1, v2)[0][0]

0.22033857

In [60]:
def word_sim(w1, w2):
    v1 = navec[w1].reshape(1, -1)
    v2 = navec[w2].reshape(1, -1)
    return cosine_similarity(v1, v2)[0][0]

In [ ]:
def sent_sim(s1, s2): 
    